In [0]:
import spacy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
drive_url = "/content/gdrive/My Drive/AI Hackathon April 2019/"
fake_url = drive_url+"fake_reviews.txt"
real_url = drive_url+"real_reviews.txt"
real_data = pd.read_csv(real_url, sep = '\n', header=None)
fake_data = pd.read_csv(fake_url, sep = '\n', header=None)

In [0]:
real_data.head()

,0
0,I bought this 4 months ago and i'm so happy wi...
1,"Quality of sound is uncanny, this is my second..."
2,I have owned two of these speakers. The first ...
3,My Mom bought these for Christmas and they are...
4,The audio is amazing and the App is a MUST HAV...


In [0]:
real_data.shape

(5000, 1)

In [0]:
real_data["Fake"] = 0
fake_data["Fake"] = 1

In [0]:
combined_labelled = pd.concat([real_data, fake_data])

In [0]:
combined_labelled.shape

(10000, 2)

In [0]:
combined_labelled.rename(columns={0:"Review", "Fake":"Fake"}, inplace=True)

In [0]:
combined_labelled.head()

,Review,Fake
0,I bought this 4 months ago and i'm so happy wi...,0
1,"Quality of sound is uncanny, this is my second...",0
2,I have owned two of these speakers. The first ...,0
3,My Mom bought these for Christmas and they are...,0
4,The audio is amazing and the App is a MUST HAV...,0


Spacy NLP to vectorize

In [0]:
nlp = spacy.load('en')

In [0]:
#combined_labelled['spacy'] = combined_labelled['Review'].apply(lambda x: nlp(x))
spacy_vector = np.matrix(combined_labelled['Review'].apply(lambda x: nlp(x).vector).tolist())

In [0]:
spacy_vector

matrix([[ 0.45501256, -0.85801744,  0.6188641 , ...,  0.13786122,
          0.13105239, -0.01498625],
        [ 0.01829837, -0.25987664,  0.84412044, ...,  0.10908449,
          0.13732207, -0.00859511],
        [ 0.17882134,  0.28101045,  0.8307099 , ...,  0.0762191 ,
          0.27353278,  0.07351179],
        ...,
        [ 0.59119433, -0.19953519,  0.00743809, ...,  0.09808144,
         -0.14909442,  0.11101654],
        [ 0.10450151,  0.01480422,  0.73532534, ...,  0.11174481,
          0.11922647,  0.02199124],
        [ 0.8010474 ,  0.52954674,  0.24215737, ...,  0.13850807,
          0.17327204,  0.03852059]], dtype=float32)

Convert Padas Series with vectorized text to matrix

In [0]:
X_train, X_test, y_train, y_test = train_test_split(spacy_vector,combined_labelled.Fake, test_size=0.33, random_state=271828)

Linear Discriminant Analysis with Spacy NLP vectorization

In [0]:
clf = LinearDiscriminantAnalysis()

In [0]:
clf.fit(X_train, y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [0]:
clf.score(X_test, y_test)

0.8909090909090909

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
vectorizer = TfidfVectorizer(min_df=1)

In [0]:
model = vectorizer.fit_transform(combined_labelled['Review'])

In [0]:
model

<10000x10926 sparse matrix of type '<class 'numpy.float64'>'
	with 269063 stored elements in Compressed Sparse Row format>

In [0]:
X_train_sk, X_test_sk, y_train_sk, y_test_sk = train_test_split(model.toarray(),combined_labelled.Fake, test_size=0.33, random_state=271828)

In [0]:
clf_sk = LinearDiscriminantAnalysis()
clf_sk.fit(X_train_sk, y_train_sk)
clf_sk.score(X_test_sk, y_test_sk)

/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


0.723030303030303

In [0]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [0]:
# SK learn vectorized data
qda_sk = QuadraticDiscriminantAnalysis()
qda_sk.fit(X_train_sk, y_train_sk)
qda_sk.score(X_test_sk, y_test_sk)

/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.6784848484848485

In [0]:
# spacy vectorized data
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)
qda.score(X_test, y_test)

0.8506060606060606

In [0]:
from sklearn import linear_model

In [0]:
sgdc = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
sgdc_sk = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)

In [0]:
sgdc.fit(X_train, y_train)
sgdc.score(X_test, y_test)

0.8884848484848484

In [0]:
sgdc_sk.fit(X_train_sk, y_train_sk)
sgdc_sk.score(X_test_sk, y_test_sk)

0.8845454545454545

Word2Vec

In [0]:
from gensim.models.word2vec import Word2Vec

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [0]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2019-04-14 09:36:26--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.0.3
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.0.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [0]:
from nltk.tokenize import TreebankWordTokenizer
from gensim.models.keyedvectors import KeyedVectors
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
word_vectors = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [0]:
def tokenize_and_vectorize(dataset):
  tokenizer = TreebankWordTokenizer()
  vectorized_data = []
  iteration = 0
  for sample in dataset:
    tokens = tokenizer.tokenize(sample)
    sample_vecs = []
    for token in tokens:
      try:
        sample_vecs.append(word_vectors[token])
      except KeyError:
        pass # No matching token in the Google w2v vocab
    vectorized_data.append(sample_vecs)
    iteration +=1
  print(iteration)
  return vectorized_data

In [0]:
def collect_expected(dataset):
  """ Peel off the target values from the dataset """
  expected = []
  for sample in dataset:
    expected.append(sample)
  return expected

In [0]:
vectorized_data = tokenize_and_vectorize(combined_labelled['Review'])
expected = collect_expected(combined_labelled.Fake)

10000


In [0]:
#X_train_v, X_test_v, y_train_v, y_test_v = train_test_split(spacy_vector, expected, test_size=0.33, random_state=271828)
X_train_v, X_test_v, y_train_v, y_test_v = train_test_split(vectorized_data, expected, test_size=0.33, random_state=271828)

In [0]:
maxlen = 300
batch_size = 32
embedding_dims = 300
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

In [0]:
def pad_trunc(data, maxlen):
  """
  For a given dataset pad with zero vectors or truncate to maxlen
  """
  new_data = []

  # Create a vector of 0s the length of our word vectors
  zero_vector = []
  for _ in range(len(data[0][0])):
    zero_vector.append(0.0)

  for sample in data:
    if len(sample) > maxlen:
      temp = sample[:maxlen]
    elif len(sample) < maxlen:
      temp = sample
      # Append the appropriate number 0 vectors to the list
      additional_elems = maxlen - len(sample)
      for _ in range(additional_elems):
        temp.append(zero_vector)
    else:
      temp = sample
    new_data.append(temp)
  return new_data

In [0]:
X_train_v = pad_trunc(X_train_v, maxlen)
X_test_v = pad_trunc(X_test_v, maxlen)

In [0]:
y_train_v = np.array(y_train_v)
y_test_v = np.array(y_test_v)

In [0]:
X_train_v = np.reshape(X_train_v, (len(X_train_v), maxlen, embedding_dims))

In [0]:
X_test_v = np.reshape(X_test_v, (len(X_test_v), maxlen, embedding_dims))

In [0]:
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Dropout, Activation, SimpleRNN, Flatten

In [0]:
model = Sequential()
model.add(Conv1D(filters,kernel_size,padding='valid',activation='relu',strides=1,input_shape=(maxlen, embedding_dims)))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.add(GlobalMaxPooling1D())

In [0]:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [0]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [0]:
del real_data
del fake_data
del combined_labelled

In [0]:
model.fit(X_train_v, y_train_v,batch_size=batch_size,epochs=epochs,validation_data=(X_test_v, y_test_v))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 6700 samples, validate on 3300 samples
Epoch 1/2
6700/6700 [==============================] - 56s 8ms/step - loss: 0.3633 - acc: 0.8321 - val_loss: 0.2676 - val_acc: 0.8833
Epoch 2/2
6700/6700 [==============================] - 54s 8ms/step - loss: 0.1477 - acc: 0.9460 - val_loss: 0.2143 - val_acc: 0.9127


In [0]:
path = F"/content/gdrive/My Drive/Hackathon/"
model_structure = model.to_json()
with open(path+"cnn_model.json", "w") as json_file:
  json_file.write(model_structure)
model.save_weights(path+"cnn_weights.h5")

In [0]:
num_neurons = 50

In [0]:
rnn_model = Sequential()
rnn_model.add(SimpleRNN(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims)))

In [0]:
rnn_model.add(Dropout(.2))
rnn_model.add(Flatten())
rnn_model.add(Dense(1, activation='sigmoid'))

In [0]:
rnn_model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])
rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 300, 50)           17550     
_________________________________________________________________
dropout_2 (Dropout)          (None, 300, 50)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 300, 50)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 15000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 15001     
Total params: 32,551
Trainable params: 32,551
Non-trainable params: 0
_________________________________________________________________


In [0]:
rnn_model.fit(X_train_v, y_train_v,batch_size=batch_size,epochs=epochs,validation_data=(X_test_v, y_test_v))

Train on 6700 samples, validate on 3300 samples
Epoch 1/2
6700/6700 [==============================] - 27s 4ms/step - loss: 0.9210 - acc: 0.6160 - val_loss: 0.4547 - val_acc: 0.7958
Epoch 2/2
6700/6700 [==============================] - 26s 4ms/step - loss: 1.1085 - acc: 0.7012 - val_loss: 0.5942 - val_acc: 0.7806


In [0]:
model_structure = rnn_model.to_json()
with open(path+"simplernn_model1.json", "w") as json_file:
  json_file.write(model_structure)
model.save_weights(path+"simplernn_weights1.h5")

In [0]:
num_neurons = 25
model = Sequential()
model.add(SimpleRNN(num_neurons, return_sequences=True, input_shape=(maxlen,embedding_dims)))
model.add(Dropout(.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_5 (SimpleRNN)     (None, 300, 25)           8150      
_________________________________________________________________
dropout_5 (Dropout)          (None, 300, 25)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 7500)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7501      
Total params: 15,651
Trainable params: 15,651
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_train_v, y_train_v,batch_size=batch_size,epochs=epochs,validation_data=(X_test_v, y_test_v))

Train on 6700 samples, validate on 3300 samples
Epoch 1/2
6700/6700 [==============================] - 20s 3ms/step - loss: 0.5799 - acc: 0.6827 - val_loss: 1.3020 - val_acc: 0.5894
Epoch 2/2
6700/6700 [==============================] - 19s 3ms/step - loss: 0.3894 - acc: 0.8315 - val_loss: 0.4558 - val_acc: 0.8142


In [0]:
from keras.layers.wrappers import Bidirectional

In [0]:
num_neurons = 10
maxlen = 300
embedding_dims = 300
model = Sequential()
model.add(Bidirectional(SimpleRNN(num_neurons, return_sequences=True),input_shape=(maxlen, embedding_dims)))
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 300, 20)           6220      
Total params: 6,220
Trainable params: 6,220
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_train_v, y_train_v,batch_size=batch_size,epochs=epochs,validation_data=(X_test_v, y_test_v))

ValueError: ignored

In [0]:
batch_size = 32
embedding_dims = 300
epochs = 2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM
num_neurons = 50
model = Sequential()
model.add(LSTM(num_neurons, return_sequences=True,input_shape=(maxlen, embedding_dims)))
model.add(Dropout(.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 300, 50)           70200     
_________________________________________________________________
dropout_7 (Dropout)          (None, 300, 50)           0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 15000)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 15001     
Total params: 85,201
Trainable params: 85,201
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.fit(X_train_v, y_train_v,batch_size=batch_size,epochs=epochs,validation_data=(X_test_v, y_test_v))

Train on 6700 samples, validate on 3300 samples
Epoch 1/2
6700/6700 [==============================] - 73s 11ms/step - loss: 0.4306 - acc: 0.7961 - val_loss: 0.4192 - val_acc: 0.8088
Epoch 2/2
6700/6700 [==============================] - 71s 11ms/step - loss: 0.2903 - acc: 0.8782 - val_loss: 0.3421 - val_acc: 0.8518


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM
num_neurons = 100
model = Sequential()
model.add(LSTM(num_neurons, return_sequences=True,input_shape=(maxlen, embedding_dims)))
model.add(Dropout(.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 300, 100)          160400    
_________________________________________________________________
dropout_9 (Dropout)          (None, 300, 100)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 30001     
Total params: 190,401
Trainable params: 190,401
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_train_v, y_train_v,batch_size=batch_size,epochs=epochs,validation_data=(X_test_v, y_test_v))

Train on 6700 samples, validate on 3300 samples
Epoch 1/2
6700/6700 [==============================] - 130s 19ms/step - loss: 0.4195 - acc: 0.8018 - val_loss: 0.3393 - val_acc: 0.8488
Epoch 2/2
6700/6700 [==============================] - 129s 19ms/step - loss: 0.2804 - acc: 0.8812 - val_loss: 0.3621 - val_acc: 0.8376


In [0]:
from keras.layers import GRU
model = Sequential()
model.add(GRU(num_neurons, return_sequences=True,input_shape=X_train_v[0].shape))

In [0]:
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 300, 100)          120300    
Total params: 120,300
Trainable params: 120,300
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_train_v, y_train_v,batch_size=batch_size,epochs=epochs,validation_data=(X_test_v, y_test_v))

ValueError: ignored

In [0]:
encoder_inputs = Input(shape=(None, input_vocab_size))
encoder = LSTM(num_neurons, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = (state_h, state_c)

NameError: ignored